## OpenWeatherMap Python API

In this notebook we will be working with [OpenWeatherMap](https://openweathermap.org/) python API.

OpenWeatherMap is a weather forecasting site, among many others. And they have exposed their data via their API so that we can interact with it using python.

## Get API Key

The first thing we need to do is get an API key. To do that, go [here](https://openweathermap.org/api) and sign up. It should only take a few minutes to do.

After that, scroll down and you see that OpenWeatherMap provides different subscription levels which provide different data. The free API key allows you access to today's weather and the 5 day forcast, among other things. This is what we will be working with today. 

OpenWeatherMap provides really nice documentation for how to submit requests using Python 


The first thing we need to do is install the 'requests' library. Requests is one of many libraries available that someone out there wrote and published for the world to use. We need to use a package manager called conda to install this. 

In [1]:
!conda install --yes requests

Solving environment: done

## Package Plan ##

  environment location: /Applications/anaconda3

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.5                |           py37_0         2.9 MB
    openssl-1.1.1h             |       haf1e3a3_0         2.2 MB
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following packages will be UPDATED:

  conda                                        4.8.4-py37_0 --> 4.8.5-py37_0
  openssl                                 1.1.1g-h1de35cc_0 --> 1.1.1h-haf1e3a3_0



conda-4.8.5          | 2.9 MB    | ##################################### | 100% 
openssl-1.1.1h       | 2.2 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Now let's import the libraries we will be using in this workshop. We don't need to install json because it is already installed with base python.

In [2]:
import json
import requests

In [7]:
#Note: this is my API key and is only being used for example purposes. Please don't spam OpenWeatherMap using my API key.
#sign up for your own API key here: https://home.openweathermap.org/users/sign_up
API_KEY = '333de4e909a5ffe9bfa46f0f89cad105'                    

#Each city in the world has a unique id number. There are over 1,000,000 so I have given you a few to start with.
#You are welcome to look in the data dictionary for more.
city_id_dict = {'Charlottesville': 4752046,                                     
                'New York': 5128581,
                'Chicago': 4887398,
                'Paris': 6455259,
                'Cape Town': 3369157,
                'Beirut': 276781,
                'Dubai': 292223,
                'Shanghai': 1796236,
                'Moscow': 524901,
                'Addis Ababa': 344979,
                'Bangkok': 1609350,
                'Oslo': 6453366,
                'Sao Paulo': 3448439,
                'Bogota': 3688689,
                'Havana': 3553478}


The following is where I create my API request via a URL. As you can see, I create a variable called "request", in which I construct a URL address which will be submitted to OpenWeatherMap. I've already created a couple variables (app_id, city_name, and city_id_string) which I insert into my url request using f-string formatting. 

Below, we are making a request for today's weather. First we will do it using the city name, next we will do it using the city_id from the city_id_dict seen above

In [40]:
city_name = 'Portland'

request = requests.get(f'http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={API_KEY}')


In [41]:
json_data = json.loads(request.text)                      #json_data is now a dictionary object
print(json_data)

{'coord': {'lon': -122.68, 'lat': 45.52}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 287.24, 'feels_like': 284.74, 'temp_min': 284.82, 'temp_max': 289.26, 'pressure': 1026, 'humidity': 76}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 310}, 'clouds': {'all': 1}, 'dt': 1601308706, 'sys': {'type': 1, 'id': 5321, 'country': 'US', 'sunrise': 1601301951, 'sunset': 1601344580}, 'timezone': -25200, 'id': 5746545, 'name': 'Portland', 'cod': 200}


Let's do this a better way! By using the city_id, which I will take from the city_id_dict object

In [44]:
city_name = 'Oslo'                                               #change the city name here
city_id_string = str(city_id_dict[f'{city_name}'])                                         

#Make a request to get today's weather. This is straight from the documentation : api.openweathermap.org/data/2.5/weather?id={city id}&appid={API key}
#request = requests.get(f'http://api.openweathermap.org/data/2.5/group?APPID={app_id}&id={city_id_string}&units=imperial')               #this actually makes the request to the API via the URL with correct parameters
request = requests.get(f'http://api.openweathermap.org/data/2.5/weather?id={city_id_string}&appid={API_KEY}&units=imperial')


*** Do It Yourself *** 

Write the previous cell again except change the city name to some other city from the city_id_dict.

Make a new API_KEY variable and include your own API key. 

Copy and paste the request line as it is an make a new API request, except it will be for a different city this time

In [13]:
# Write your code here





I now take the request and load the response, which is JSON data. Let's take a look at the weather results below for your chosen city.

In [17]:
json_data = json.loads(request.text)                      #json_data is now a dictionary object
print(json_data)



{'coord': {'lon': 10.73, 'lat': 59.91}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 58.01, 'feels_like': 56.14, 'temp_min': 57, 'temp_max': 59, 'pressure': 991, 'humidity': 72}, 'visibility': 10000, 'wind': {'speed': 3, 'deg': 74}, 'clouds': {'all': 24}, 'dt': 1601307011, 'sys': {'type': 3, 'id': 2009047, 'country': 'NO', 'sunrise': 1601270163, 'sunset': 1601312347}, 'timezone': 7200, 'id': 6453366, 'name': 'Oslo', 'cod': 200}


See above that the data returned is simply a dictionary and we can parse that just like we would any other dictionary object.

In [18]:
for keys, values in json_data.items():
    print(keys)                                       #all the interesting data is contained in the 'list' key


coord
weather
base
main
visibility
wind
clouds
dt
sys
timezone
id
name
cod


Inside the 'weather' key is a list of data. Lists are ordered objects, so we can use indexing to look more closely at each item in the list.

In [22]:
print(json_data['weather'])                    #Looking at data in the 'weather' key of the json_data dictionary


[{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}]


And again, inside that list object we have a dictionary, which we can parse the same as before. The actual conditions themselves are in the 'main' and 'description' keys.

In [24]:
print(json_data['weather'][0]['main'])
print(json_data['weather'][0]['description'])

Clouds
few clouds


Another dictionary you might want to investigate from the json_data object is the 'main' dictionary

In [25]:
print(json_data['main'])

{'temp': 58.01, 'feels_like': 56.14, 'temp_min': 57, 'temp_max': 59, 'pressure': 991, 'humidity': 72}


Or if this gets confusing and unreadable, you can make a variable and store this more cleanly

*** Do It Yourself ***

Print the current temperature today the city that you chose

Print the minimum and maximum temperature

In [26]:
# Write your code here






*** Do It Yourself ***

Choose one of the other keys from the json_data object. Investigate one that might be interesting to you and see what is inside!

In [27]:
# Write your code here






Now lets make a request to get the 5 day forecast. This returns the forecast at 3 hour intervals for the next 5 days. Again, I took the 'request' straight from the documentation page for OpenWeatherMap

<strong> Note: </strong> The 5 day forecast appears not to be documented any more in OpenWeatherMap's API documentation? The docs now list a '4 day weather forecast' option. But I liked the 5 day weather forecast example I have used in the past so I stuck with it. My guess is that this example will eventually stop working, but OpenWeatherMap maintains it as a legacy tool because there are probably a lot of people still using this API in their applications (myself included!)

In [51]:
#make request for 5 day forecast
request = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?id={city_id_string}&APPID=333de4e909a5ffe9bfa46f0f89cad105&units=imperial&')

json_data = json.loads(request.text)

print(json_data)            #note how messy and hard to read this is. Yet, it is a dictionary


{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1601316000, 'main': {'temp': 56.91, 'feels_like': 55.47, 'temp_min': 56.82, 'temp_max': 56.91, 'pressure': 1003, 'sea_level': 1003, 'grnd_level': 1014, 'humidity': 78, 'temp_kf': 0.05}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'clouds': {'all': 30}, 'wind': {'speed': 2.77, 'deg': 32}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2020-09-28 18:00:00'}, {'dt': 1601326800, 'main': {'temp': 57.09, 'feels_like': 55.96, 'temp_min': 57.09, 'temp_max': 57.11, 'pressure': 1010, 'sea_level': 1010, 'grnd_level': 1015, 'humidity': 81, 'temp_kf': -0.01}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 83}, 'wind': {'speed': 2.8, 'deg': 39}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2020-09-28 21:00:00'}, {'dt': 1601337600, 'main': {'temp': 56.5, 'feels_like': 55.54, 'temp_min': 56.46, 'temp_ma

This is messy, so let's separate each 3 hour forecast with a blank line. 

There is still a lot of data. How much exactly?  24 hours / 3 hours = 8. 5 days x 8 predictions per day = 40 predictions total.

In [54]:
#take my word for it, the interesting stuff is in the 'list' key of the json_data dictionary

for measurement in json_data['list']:
    print(measurement)
    print()

{'dt': 1601316000, 'main': {'temp': 56.91, 'feels_like': 55.47, 'temp_min': 56.82, 'temp_max': 56.91, 'pressure': 1003, 'sea_level': 1003, 'grnd_level': 1014, 'humidity': 78, 'temp_kf': 0.05}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'clouds': {'all': 30}, 'wind': {'speed': 2.77, 'deg': 32}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2020-09-28 18:00:00'}

{'dt': 1601326800, 'main': {'temp': 57.09, 'feels_like': 55.96, 'temp_min': 57.09, 'temp_max': 57.11, 'pressure': 1010, 'sea_level': 1010, 'grnd_level': 1015, 'humidity': 81, 'temp_kf': -0.01}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 83}, 'wind': {'speed': 2.8, 'deg': 39}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2020-09-28 21:00:00'}

{'dt': 1601337600, 'main': {'temp': 56.5, 'feels_like': 55.54, 'temp_min': 56.46, 'temp_max': 56.5, 'pressure': 1013, 'sea_level': 1013, 'g

If I was not sure how many items were in this list, I can print it using the python built-in len() function

In [55]:
print(len(json_data['list']))

40


*** Do It Yourself ***

Print the datetime stamp for each measurement


In [63]:
# Write your code here. Start with this...

for measurement in json_data['list']:
    print(measurement)
    print()

2020-09-28 18:00:00
2020-09-28 21:00:00
2020-09-29 00:00:00
2020-09-29 03:00:00
2020-09-29 06:00:00
2020-09-29 09:00:00
2020-09-29 12:00:00
2020-09-29 15:00:00
2020-09-29 18:00:00
2020-09-29 21:00:00
2020-09-30 00:00:00
2020-09-30 03:00:00
2020-09-30 06:00:00
2020-09-30 09:00:00
2020-09-30 12:00:00
2020-09-30 15:00:00
2020-09-30 18:00:00
2020-09-30 21:00:00
2020-10-01 00:00:00
2020-10-01 03:00:00
2020-10-01 06:00:00
2020-10-01 09:00:00
2020-10-01 12:00:00
2020-10-01 15:00:00
2020-10-01 18:00:00
2020-10-01 21:00:00
2020-10-02 00:00:00
2020-10-02 03:00:00
2020-10-02 06:00:00
2020-10-02 09:00:00
2020-10-02 12:00:00
2020-10-02 15:00:00
2020-10-02 18:00:00
2020-10-02 21:00:00
2020-10-03 00:00:00
2020-10-03 03:00:00
2020-10-03 06:00:00
2020-10-03 09:00:00
2020-10-03 12:00:00
2020-10-03 15:00:00


*** Do It Yourself ***

Print just the datetime stamp if the time is midnight

In [66]:
# Write your code here. Start with this...


for measurement in json_data['list']:
    if measurement['dt_txt'][11:20] == '00:00:00':
        print(measurement['dt_txt'])


2020-09-29 00:00:00
2020-09-30 00:00:00
2020-10-01 00:00:00
2020-10-02 00:00:00
2020-10-03 00:00:00


Just as an example, let's use string indexing to find one forecast prediction for each day. Luckily, this data is structured so look at the below example. From looking at the data, in the 'dt_txt' key I have a standard date/time stamp. You can see below I am taking the measurement at just 12 noon for each day. The time stamp is at the locations 11-20 in each 'dt_txt' key.

After doing this, I just have 5 results. One for each of the next five days for my target city.

In [56]:
for daily_measurement in json_data['list']:
    if daily_measurement['dt_txt'][11:20] == '12:00:00':
        print(daily_measurement)
        print(daily_measurement['dt_txt'][:10])
        print(daily_measurement['main']['temp'])                                   #it gets tricky to parse through nested dictionaries
        print()

{'dt': 1601380800, 'main': {'temp': 55.58, 'feels_like': 54.25, 'temp_min': 55.58, 'temp_max': 55.58, 'pressure': 1015, 'sea_level': 1015, 'grnd_level': 1015, 'humidity': 88, 'temp_kf': 0}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': {'all': 99}, 'wind': {'speed': 3.53, 'deg': 159}, 'visibility': 10000, 'pop': 0.86, 'rain': {'3h': 0.5}, 'sys': {'pod': 'd'}, 'dt_txt': '2020-09-29 12:00:00'}
2020-09-29
55.58

{'dt': 1601467200, 'main': {'temp': 57.06, 'feels_like': 55, 'temp_min': 57.06, 'temp_max': 57.06, 'pressure': 1016, 'sea_level': 1016, 'grnd_level': 1016, 'humidity': 79, 'temp_kf': 0}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'clouds': {'all': 97}, 'wind': {'speed': 4.09, 'deg': 160}, 'visibility': 10000, 'pop': 0.11, 'sys': {'pod': 'd'}, 'dt_txt': '2020-09-30 12:00:00'}
2020-09-30
57.06

{'dt': 1601553600, 'main': {'temp': 57.97, 'feels_like': 53.8, 'temp_min': 57.97, 'temp_max

Here, I use some string formatting and a little more filtering to print some nice looking output. Here, I have found the time stamp of 12 noon for each day. Then within that I have taken the date from the date/time stamp. Then the temperature for each day. And finally put it all together in a string, printing the city, date, and temperature for that day.

In [57]:
#A cleaner version of the above block
for daily_measurement in json_data['list']:
    time_stamp = daily_measurement['dt_txt'][11:20]
    if time_stamp == '12:00:00':
        day = daily_measurement['dt_txt'][:10]
        temperature = daily_measurement['main']['temp']
        print(f'The temperature in {city_name} on {day} will be {temperature} degrees')

The temperature in Oslo on 2020-09-29 will be 55.58 degrees
The temperature in Oslo on 2020-09-30 will be 57.06 degrees
The temperature in Oslo on 2020-10-01 will be 57.97 degrees
The temperature in Oslo on 2020-10-02 will be 58.77 degrees
The temperature in Oslo on 2020-10-03 will be 56.89 degrees


So as you see, this allows you to slice and dice the weather data in many ways. You can automate things, depending on weather conditions, link this to other applications, and so on. I encourange you to play with the script, change the city, look at the results, and so on. 